In [15]:
import pandas as pd
import utils
import plotly.graph_objects as go

In [16]:
pair = "EUR_USD"
granularity = "H1"
ma_list = [16,64]

In [42]:
df = pd.read_pickle(utils.get_his_data_filename(pair, granularity))

In [18]:
non_cols = ['time', 'volume']
mod_cols = [x for x in df.columns if x not in non_cols]
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [19]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()

In [20]:
for ma in ma_list:  
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()

In [21]:
df_ma.dropna(inplace=True)

In [44]:
df_ma.reset_index(drop=True, inplace=True)

In [45]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE
0,2020-06-11T10:00:00.000000000Z,1.13835,1.13854,1.13702,1.13724,1.137317,1.133934,0.003383,NaN,False
1,2020-06-11T11:00:00.000000000Z,1.13726,1.13750,1.13534,1.13558,1.137107,1.134026,0.003081,0.003383,False
2,2020-06-11T12:00:00.000000000Z,1.13558,1.13586,1.13365,1.13567,1.136992,1.134123,0.002868,0.003081,False
3,2020-06-11T13:00:00.000000000Z,1.13568,1.13788,1.13508,1.13779,1.137001,1.134258,0.002743,0.002868,False
4,2020-06-11T14:00:00.000000000Z,1.13780,1.14036,1.13720,1.13813,1.137024,1.134394,0.002631,0.002743,False


In [23]:
df_ma['DIFF'] = df_ma.MA_16 - df_ma.MA_64

In [25]:
df_ma['DIFF_PREV'] = df_ma.DIFF.shift(1)

In [27]:
def is_trade(row):
    if row.DIFF >= 0 and row.DIFF_PREV < 0:
        return True
    if row.DIFF <= 0 and row.DIFF_PREV > 0:
        return True
    return False

In [28]:
df_ma['IS_TRADE'] = df_ma.apply(is_trade, axis=1)

In [46]:
df_trades = df_ma[df_ma.IS_TRADE==True].copy()

In [47]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE
12,2020-06-11T22:00:00.000000000Z,1.12936,1.12988,1.12874,1.12988,1.134564,1.134750,-0.000186,0.000247,True
62,2020-06-16T00:00:00.000000000Z,1.13378,1.13400,1.13304,1.13337,1.129281,1.129236,0.000046,-0.000461,True
84,2020-06-16T22:00:00.000000000Z,1.12620,1.12624,1.12535,1.12593,1.128306,1.128339,-0.000033,0.000441,True
174,2020-06-22T16:00:00.000000000Z,1.12600,1.12640,1.12572,1.12611,1.121703,1.121438,0.000265,-0.000208,True
227,2020-06-24T21:00:00.000000000Z,1.12528,1.12570,1.12516,1.12550,1.127857,1.127885,-0.000027,0.000437,True


In [48]:
df_plot = df_ma.iloc[0:24].copy()

In [49]:
df_plot.shape

(24, 10)

In [50]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor='#24A06B',
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color='#2EC886',  
    decreasing_line_color='#FF3A4C'
))
for ma in ma_list:  
    col = f"MA_{ma}"
    fig.add_trace(go.Scatter(x=df_plot.time, 
        y=df_plot[col],
        line=dict(width=2),
        line_shape='spline',
        name=col
        ))
fig.update_layout(width=1000,height=400,
    margin=dict(l=10,r=10,b=10,t=10),
    font=dict(size=10,color="#e1e1e1"),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True
)
fig.show()